In [1]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB
from gurobipy import quicksum
import gurobipy as gp
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MultipleLocator
import matplotlib
from scipy.stats import norm
from scipy.linalg import sqrtm
from tqdm import tqdm
import scipy.sparse as sp
import pickle
import os

In [2]:
color_platte = ['#93003a', '#00429d', '#93c4d2', '#6ebf7c']
matplotlib.rcParams.update({'legend.fontsize': 14, 'legend.handlelength': 2})
sns.set(style ="whitegrid", font_scale=1.5)

### excel数据处理

In [3]:
import importlib
import utils
importlib.reload(utils)
from utils import calc_cum_arrive, calc_cum_depart #, calc_wasted_capacity
cap = 900

In [4]:
df = pd.read_excel('BCN.xlsx').iloc[:260,:]
df['Time'] = pd.to_timedelta(df['Time'].astype(str))
df['minutes'] = df['Time'].dt.total_seconds() / 60
df['slots'] = df['minutes'] // 15
df = df[['Aircraft Capacity', 'slots']] 

# Passengers Arrive An Hour Prior
df['slots'] = df['slots'].apply(lambda x: x-4)

In [5]:
full = pd.DataFrame({'slots':np.arange(24*4), 'capacity': cap})
full = pd.merge(full, df, on='slots', how='left')
full = full.fillna(0)
full = full.groupby('slots').agg({'capacity':'first', 'Aircraft Capacity':'sum'}).reset_index()
dt = []
vt = full['Aircraft Capacity'].cumsum()
total_departures = 0
arrival_queue = 0
for row in full.iterrows():
    arrival_queue += row[1]['Aircraft Capacity']
    departed = min(cap, arrival_queue)
    arrival_queue -= departed
    total_departures += departed
    dt.append(total_departures)
dt = np.array(dt)
expanded = []
for row in full.iterrows():
    expanded += [row[1]['slots'] for i in range(int(row[1]['Aircraft Capacity']))]

In [6]:
full

,slots,capacity,Aircraft Capacity
0,0,900,0.0
1,1,900,0.0
2,2,900,288.0
3,3,900,0.0
4,4,900,0.0
...,...,...,...
91,91,900,0.0
92,92,900,0.0
93,93,900,0.0
94,94,900,0.0


In [7]:
def process(full, nJ=96):

    da = []
    aft = []
    nI = full['Aircraft Capacity'].sum().astype(int)
    c = np.zeros([nI, nJ])
    beta = 2
    for i in range(nI):
        lead_time = 4
        for j in range(nJ):
            desired_arrival = expanded[i] 
            actual_flight_time = expanded[i]+lead_time
            
            if j < desired_arrival:
                cost = abs(j - desired_arrival)*beta
            else:
                cost = 200
            c[i, j] = cost
        aft.append(actual_flight_time)
        da.append(desired_arrival)

    return c, np.array(aft), np.array(da)

c, aft, da = process(full)
aft = calc_cum_arrive(aft)

In [10]:
# chance constraint
# 先假定乘客只有10人
pax = 10
m = Model('Assignment')
x = m.addVars(pax, c.shape[1], vtype=GRB.BINARY, name='x')
rhs = m.addVars(c.shape[1], name="rhs_j")

def robust(full):
    std_pct: float = 0.2
    chance_beta: float = 0.05
    corr = 0.2 * np.random.rand(2*pax, 2*pax)
    alpha = np.array([0.8] * pax) # 待定
    vector1 = np.array([1] * pax)
    std = np.concatenate((std_pct * alpha, vector1), axis=0)
    sigma_half = np.diag(std) @ np.real(sqrtm(corr)) # 待定

    # 生成A矩阵
    base_matrix = np.array([
    [1, -1/2, -1/2],
    [0,  1/2,  1/2]
    ])
    identity_block = sp.eye(pax)
    A_mat = sp.kron(base_matrix, identity_block)

    ppf_beta = norm.ppf(1 - chance_beta)

    for j in tqdm(range(c.shape[1])):

        if j == 0:
            m.addConstr(
            (
                rhs[j] == 1/ppf_beta * (
                    cap - (
                        quicksum(alpha[i]*x[i,j] for i in range(pax)) - 
                        1/2 * quicksum(alpha[i]*0 for i in range(pax)) - 
                        1/2 * quicksum(alpha[i]*x[i,j+1] for i in range(pax)) +
                        1/2 * quicksum(0 for i in range(pax)) +
                        1/2 * quicksum(x[i,j+1] for i in range(pax))
                    )
                )
            ),
            "robust_constraint_{j=0}",
        )

        elif j == c.shape[1] - 1:
            m.addConstr(
            (
                rhs[j] == 1/ppf_beta * (
                    cap - (
                        quicksum(alpha[i]*x[i,j] for i in range(pax)) - 
                        1/2 * quicksum(alpha[i]*x[i,j-1] for i in range(pax)) - 
                        1/2 * quicksum(alpha[i]*0 for i in range(pax)) +
                        1/2 * quicksum(x[i,j-1] for i in range(pax)) +
                        1/2 * quicksum(0 for i in range(pax))
                    )
                )
            ),
            "robust_constraint_{j=c.shape[1] - 1}",
        )
        
        else:
            m.addConstr(
                (
                    rhs[j] == 1/ppf_beta * (
                        cap - (
                            quicksum(alpha[i]*x[i,j] for i in range(pax)) - 
                            1/2 * quicksum(alpha[i]*x[i,j-1] for i in range(pax)) - 
                            1/2 * quicksum(alpha[i]*x[i,j+1] for i in range(pax)) +
                            1/2 * quicksum(x[i,j-1] for i in range(pax)) +
                            1/2 * quicksum(x[i,j+1] for i in range(pax))
                        )
                    )
                ),
                "robust_constraint_{j}",
            )
            
        x_mat = m.addMVar((3*pax,), name=f'x_mat_j_{j}')
        m.addConstrs((x_mat[i] == x[i, j] for i in range(pax)), f"X_MVar1_j_{j}")
        if j > 0:
            m.addConstrs((x_mat[pax + i] == x[i, j - 1] for i in range(pax)), f"X_MVar2_j_{j}")
        else:
            m.addConstrs((x_mat[pax + i] == 0 for i in range(pax)), f"X_MVar2_j_{j}")

        if j < c.shape[1] - 1:
            m.addConstrs((x_mat[2*pax + i] == x[i, j + 1] for i in range(pax)), f"X_MVar3_j_{j}")
        else:
            m.addConstrs((x_mat[2*pax + i] == 0 for i in range(pax)), f"X_MVar3_j_{j}")

        x_sigma = m.addMVar((2*pax,))
        m.addConstr((x_sigma == sigma_half @ A_mat @ x_mat), "cc_energy0")
        m.addConstr(
            (x_sigma @ x_sigma <= rhs[j] * rhs[j]),
            "cc_energy1",
        )
    return None

robust(full)

100%|██████████| 96/96 [00:00<00:00, 117.32it/s]


### Run

In [11]:
for i in tqdm(range(pax)):
    m.addConstr(quicksum(x[i,j] for j in range(c.shape[1])) == 1)
for j in tqdm(range(c.shape[1])):
    m.addConstr(quicksum(x[i,j] for i in range(pax)) <= cap)

m.setObjective(quicksum(x[i,j]*c[i,j] for i in range(pax) for j in range(c.shape[1])), GRB.MINIMIZE)
m.update()
m.optimize()

100%|██████████| 96/96 [00:00<00:00, 95755.81it/s]

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-14900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 5002 rows, 5856 columns and 70136 nonzeros
Model fingerprint: 0xe582a5c4
Model has 96 quadratic constraints
Variable types: 4896 continuous, 960 integer (960 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 3072 rows and 2976 columns
Presolve time: 0.14s
Presolved: 1930 rows, 2880 columns, 60080 nonzeros
Variable types: 1920 continuous, 960 integer (960 binary)

Root relaxation: objective 1.906636e+02, 188 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

     0     0  267.24860    0   81          -  267.24860      -     -    0s
     0     0  341.73195    0   82          -  341.73195      -     -    0s
     0     0  341.73195    0   81          -  341.73195      -     -    0s
     0     0  341.73195    0   81          -  341.73195      -     -    0s
     0     2  493.91560    0   81          -  493.91560      -     -    1s
   250     8 1173.70028   23  164          - 1173.06045      -   372    5s
   288     8 1584.56933   28  205          - 1565.11327      -   605   10s
   641    52 2000.00000   46  273          - 2000.00000      -   594   15s
  1056     6 2000.00000   63  237          - 2000.00000      -   569   24s
  1060     6 2000.00000   64  278          - 2000.00000      -   659   39s
  1066     8 2000.00000   65  291          - 2000.00000      -   711   42s
  1149    16 2000.00000   72  116          - 2000.00000      -   743   49s
  1168     9 2000.00000   73  252          - 2000.00000      -   784   50s
  1231    13 infeasible  